In [1]:
from typing import Tuple, List
import polars as pl
from pprint import pprint
import sys
from pathlib import Path
import psutil
import pandas as pd
import time
import numpy as np

In [2]:
df = pl.scan_parquet("data/scan_preprocess.parquet")  # LazyFrame

In [3]:
total_rows = df.select(pl.len()).collect().item()
total_cols = len(df.collect_schema().names())
print(f"전체 행: {total_rows:,}개, 전체 컬럼: {total_cols}개")

전체 행: 2,247,150개, 전체 컬럼: 357개


In [4]:
dedup_cols = [
    'report_number',
    'date_of_event', 
    'device_0_manufacturer_d_name',
    'device_0_udi_di',
    'device_0_lot_number',
    'device_0_udi_public'
]

# Unknown / N/A 패턴 리스트
na_patterns = r'^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$'

In [5]:
# 일단 중복된 행들만 확인
# 조합 (6개 컬럼)의 개수
df_with_cnt = df.with_columns(
    pl.len().over(dedup_cols).alias('duplicate_cnt')
)

# cnt가 2 이상이 경우에만 
# cnt가 1인 경우에는 삭제할 필요가 없으므로
df_duplicates_only = df_with_cnt.filter(
    pl.col('duplicate_cnt') >= 2
)

duplicate_cnt = df_duplicates_only.select(pl.len()).collect().item()
print(f"중복된 행의 개수: {duplicate_cnt:,}개")

unique_cnt = df.unique(subset=dedup_cols, maintain_order=True).select(pl.len()).collect().item()
print(f"유일한 행의 개수: {unique_cnt:,}개")

중복된 행의 개수: 13,254개
유일한 행의 개수: 2,239,672개


In [6]:
sample = df_duplicates_only.select(
    dedup_cols + ['duplicate_cnt']
).head(10).collect()

print(sample)

shape: (10, 7)
┌──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│ report_numbe ┆ date_of_eve ┆ device_0_ma ┆ device_0_ud ┆ device_0_lo ┆ device_0_ud ┆ duplicate_c │
│ r            ┆ nt          ┆ nufacturer_ ┆ i_di        ┆ t_number    ┆ i_public    ┆ nt          │
│ ---          ┆ ---         ┆ d_name      ┆ ---         ┆ ---         ┆ ---         ┆ ---         │
│ str          ┆ date        ┆ ---         ┆ str         ┆ str         ┆ str         ┆ u32         │
│              ┆             ┆ str         ┆             ┆             ┆             ┆             │
╞══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│ 0009613348-2 ┆ 2024-02-14  ┆ INSTITUT    ┆ null        ┆ ARM66       ┆ null        ┆ 2           │
│ 024-006866   ┆             ┆ STRAUMANN   ┆             ┆             ┆             ┆             │
│              ┆             ┆ AG          ┆             ┆             ┆    

# 함수 정의

In [7]:
def remove_na_values(df, dedup_cols, na_patterns, verbose=True):
    """
    Na / Unknwon 값이 있는 행을 제거하는 함수
    
    작동방식 :
    1. 각 컬럼에 대해 유효한 값인지 체크
    2. 모든 조건은 포함한(모두 만족하는) 행으로 필터링
    3. 필터 적용

    Parameters:
    df : polars.DataFrame
        원본 DF
    dedup_cols : list
        -> 모두 유효해야 함
    na_patterns : str
        NA / Unknown 패턴 정규식
    verbose : bool
        -> 현재 진행상황 출력해서 확인할 수 있게

    Returns: 
    polars.LazyFrame
        비어진 값이나 모르는 값이 없는 DF
    """

    # 진행상확 확인
    if verbose:
        print("na 값 제거")
        print(f"패턴: {na_patterns}")

    # 제거 되기 전 개수 확인
    before_cnt = df.select(pl.len()).collect().item()
    if verbose:
        print(f"제거 전 행 개수 : {before_cnt:,}개")
    
    # ===
    # 각 컬럼별로 필터 조건
    # ===
    conditions = []

    for col in dedup_cols:
        # 컬럼이 존재하는지 확인
        if col in df.collect_schema().names():
            # 유효한 값의 조건
            # null / na_patterns 패턴에 매칭되지 않는 값
            cond = (
                pl.col(col).is_not_null()
                &
                ~ pl.col(col).cast(pl.Utf8).str.to_uppercase().str.contains(na_patterns)
                # ~ 은 not 연산자
            )
            conditions.append(cond)

            if verbose:
                print(f"컬럼 '{col}'에 대해 na/unknown 값 제거 조건 추가")
        else:
            if verbose:
                print(f"컬럼 '{col}'이(가) 존재하지 않음. 건너뜀")
        
# 예외처리
    if not conditions:
        if verbose:
            print("제거할 조건이 없음. 원본 DF 반환")
        return df

# ===
# 모든 조건을 and 조건으로 결합
# (모든 조건을 만족해야 함)
# ===
    final_condition = conditions[0]
    for cond in conditions[1:]:
        final_condition = final_condition & cond

# 필터 적용
    df_cleaned = df.filter(final_condition)

# 결과
    after_cnt = df_cleaned.select(pl.len()).collect().item()
    removed_cnt = before_cnt - after_cnt

    if verbose:
        print(f"제거 후 행 개수 : {after_cnt:,}개")
        print(f"제거된 행 개수 : {removed_cnt:,}개")

        return df_cleaned

In [8]:
def analyze_duplicates(df, group_cols, verbose = True):
    """
    중복 데이터 분석 함수들

    작동 방식 
    1. 전체 개수 확인
    2. 고유(unique) 개수 확인
    3. 전체 - 고유 = 중복 개수 확인

    Parameters:
    df : polars.DataFrame -> 원본
    dedup_cols : list -> 중복 확인 컬럼
    verbose : bool -> 진행상황 출력 여부

    returns:

    tuple : (전체 개수, 고유 개수, 중복 개수)
    """

    if verbose:
        print(f" 중복 확인")

    # 전체 개수
    total_cnt = df.select(pl.len()).collect().item()

    # 고유 개수
    unique_cnt = df.unique(
        subset = group_cols, # 중복 ㅎ판단
        maintain_order = True
    ).select(pl.len()).collect().item()

    # 중복 개수
    duplicate_cnt = total_cnt - unique_cnt

    if verbose:
        print(f"전체 개수 : {total_cnt:,}개")
        print(f"고유 개수 : {unique_cnt:,}개")
        print(f"중복 개수 : {duplicate_cnt:,}개")
        for i, col in enumerate(dedup_cols, start=1):
            print(f"{i}. {col}")

    return total_cnt, unique_cnt, duplicate_cnt

In [9]:
def remove_duplicates(df, dedup_cols, keep = 'first', verbose = True):
    """
    중복 데이터 제거 함수

    작동 방식
    1. dedup_cols 기준으로 중복 판단
    2. keep 옵션에 따라 첫번째/마지막 행 유지
    3. 중복 제거된 DF 반환

    Parameters:
    df : polars.DataFrame -> 원본 DF
    dedup_cols : list -> 중복 판단 컬럼 리스트
    keep : str -> 'first' or 'last'
        'first' : 첫번째 행 유지
        'last' : 마지막 행 유지
    verbose : bool -> 진행상황 출력 여부

    Returns:
    polars.DataFrame -> 중복 제거된 DF
    """

    if verbose:
        print("중복 제거 시작")
        print(f"중복 판단 컬럼: {dedup_cols}")
        print(f"유지 옵션: {keep}")

    # 중복 제거
    df_deduped = df.unique(
        subset = dedup_cols,
        maintain_order = True,
        keep = 'first'
    )

    if verbose:
        before_cnt = df.select(pl.len()).collect().item()
        after_cnt = df_deduped.select(pl.len()).collect().item()
        removed_cnt = before_cnt - after_cnt

        print(f"제거 전 행 개수 : {before_cnt:,}개")
        print(f"제거 후 행 개수 : {after_cnt:,}개")
        print(f"제거된 행 개수 : {removed_cnt:,}개")

    return df_deduped

## 함수 실행

In [10]:
#df_cleaned = remove_na_values(df_duplicates_only, dedup_cols, na_patterns)
#df_cleaned

In [11]:
#total, unique, duplicate = analyze_duplicates(df_cleaned, dedup_cols)
#pprint((total, unique, duplicate))  

In [12]:
#df_final = remove_duplicates(df_cleaned, dedup_cols, keep='first')
#df_final

In [13]:
# 완전한 파이프라인
df_all_cleaned = remove_na_values(df, dedup_cols, na_patterns)
total, unique, dup = analyze_duplicates(df_all_cleaned, dedup_cols)
df_all_final = remove_duplicates(df_all_cleaned, dedup_cols, keep='first')


na 값 제거
패턴: ^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$
제거 전 행 개수 : 2,247,150개
컬럼 'report_number'에 대해 na/unknown 값 제거 조건 추가
컬럼 'date_of_event'에 대해 na/unknown 값 제거 조건 추가
컬럼 'device_0_manufacturer_d_name'에 대해 na/unknown 값 제거 조건 추가
컬럼 'device_0_udi_di'에 대해 na/unknown 값 제거 조건 추가
컬럼 'device_0_lot_number'에 대해 na/unknown 값 제거 조건 추가
컬럼 'device_0_udi_public'에 대해 na/unknown 값 제거 조건 추가
제거 후 행 개수 : 825,780개
제거된 행 개수 : 1,421,370개
 중복 확인
전체 개수 : 825,780개
고유 개수 : 822,738개
중복 개수 : 3,042개
1. report_number
2. date_of_event
3. device_0_manufacturer_d_name
4. device_0_udi_di
5. device_0_lot_number
6. device_0_udi_public
중복 제거 시작
중복 판단 컬럼: ['report_number', 'date_of_event', 'device_0_manufacturer_d_name', 'device_0_udi_di', 'device_0_lot_number', 'device_0_udi_public']
유지 옵션: first
제거 전 행 개수 : 825,780개
제거 후 행 개수 : 822,738개
제거된 행 개수 : 3,042개


In [14]:
row_count = df_all_final.select(pl.len()).collect().item()
print(f"행 수: {row_count}")

행 수: 822738


In [15]:
col_count = len(df_all_final.columns)
print(f"열 수: {col_count}")

열 수: 357


C:\Users\USER\AppData\Local\Temp\ipykernel_14204\1450529416.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  col_count = len(df_all_final.columns)


# device_class3인 것만 필터링

In [16]:
# device_class가 3인 컬럼 찾기
device_class_cols = [
    col for col in df_all_final.columns 
    if 'device_' in col and 'openfda_device_class' in col
]

print(f"찾은 컬럼들: {device_class_cols}")
# ['device_0_openfda_device_class', 'device_1_openfda_device_class', ...]

# 하나라도 3이면 필터링
condition = pl.lit(False)  # 초기값 False
for col in device_class_cols:
    condition = condition | (pl.col(col) == "3")

df_class3 = df_all_final.filter(condition)

찾은 컬럼들: ['device_0_openfda_device_class', 'device_1_openfda_device_class']


C:\Users\USER\AppData\Local\Temp\ipykernel_14204\3284298869.py:3: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  col for col in df_all_final.columns


In [17]:
# "Class 3 기기와 관련된 모든 사건"
df_class3 = df_all_final.filter(
    (pl.col('device_0_openfda_device_class') == "3") |
    (pl.col('device_1_openfda_device_class') == "3")
)

In [18]:
row_count = df_class3.select(pl.len()).collect().item()
print(f"행 수: {row_count}")

행 수: 186762


In [19]:
col_count = len(df_class3.columns)
print(f"열 수: {col_count}")

열 수: 357


C:\Users\USER\AppData\Local\Temp\ipykernel_14204\1618944189.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  col_count = len(df_class3.columns)


## device_class3인 것 event_type 분포 확인

In [20]:
# NULL 포함 확인
event_dist = df_class3.group_by('event_type').agg([
    pl.len().alias('count')
]).with_columns([
    (pl.col('count') / pl.col('count').sum() * 100).round(2).alias('percentage')
]).sort('count', descending=True).collect()

# NULL 개수 별도 확인
null_count = df_class3.filter(
    pl.col('event_type').is_null()
).select(pl.len()).collect().item()

print(event_dist)
print(f"\nNULL 값: {null_count:,}개")

shape: (3, 3)
┌─────────────┬────────┬────────────┐
│ event_type  ┆ count  ┆ percentage │
│ ---         ┆ ---    ┆ ---        │
│ cat         ┆ u32    ┆ f64        │
╞═════════════╪════════╪════════════╡
│ Malfunction ┆ 100820 ┆ 53.98      │
│ Injury      ┆ 82253  ┆ 44.04      │
│ Death       ┆ 3689   ┆ 1.98       │
└─────────────┴────────┴────────────┘

NULL 값: 0개


## 이상기기 topN

In [21]:
total = df_class3.select(pl.len()).collect().item()

top_n = 20  # 원하는 개수

top_devices = df_class3.group_by('device_0_generic_name').agg([
    pl.len().alias('count')
]).with_columns([
    (pl.col('count') / total * 100).round(2).alias('percentage')
]).sort('count', descending=True).head(top_n).collect()

print(f"=== Class 3 기기 사건 수 Top {top_n} (전체: {total:,}건) ===\n")

for i, row in enumerate(top_devices.iter_rows(named=True), 1):
    device = row['device_0_generic_name'] if row['device_0_generic_name'] else "(NULL)"
    count = row['count']
    pct = row['percentage']
    print(f"{i:2d}. {device[:70]:70s} {count:>8,}건 ({pct:>5.1f}%)")

=== Class 3 기기 사건 수 Top 20 (전체: 186,762건) ===

 1. AUTOMATED INSULIN DOSING DEVICE SYSTEM, SINGLE HORMONAL CONTROL          56,471건 ( 30.2%)
 2. TEMPORARY NON-ROLLER TYPE LEFT HEART SUPPORT BLOOD PUMP                  13,250건 (  7.1%)
 3. PUMP, INFUSION, INSULIN, TO BE USED WITH INVASIVE GLUCOSE SENSOR         11,326건 (  6.1%)
 4. PROSTHESIS, BREAST, NONINFLATABLE, INTERNAL, SILICONE GEL-FILLED          7,781건 (  4.2%)
 5. DEVICE, HEMOSTASIS, VASCULAR                                              6,164건 (  3.3%)
 6. STIMULATOR, SPINAL-CORD, TOTALLY IMPLANTED FOR PAIN RELIEF                5,726건 (  3.1%)
 7. AUTOMATED INSULIN DOSING, THRESHOLD SUSPEND                               5,230건 (  2.8%)
 8. IMPLANTABLE LEAD                                                          5,045건 (  2.7%)
 9. SCS IPG                                                                   5,022건 (  2.7%)
10. SCS LEAD                                                                  4,842건 (  2.6%)
11. NO MATCH 

## top3 기기의 event_type 분포 확인

In [22]:

top3 = df_class3.group_by('device_0_generic_name').len()\
    .sort('len', descending=True).head(3).collect()

print("=" * 90)
print("Top 3 기기별 Event Type 분포")
print("=" * 90)

for rank, row in enumerate(top3.iter_rows(named=True), 1):
    device = row['device_0_generic_name']
    total = row['len']
    
    print(f"\n[{rank}위] {device}")
    print(f"총 사건 수: {total:,}건")
    print("-" * 90)
    
    dist = df_class3.filter(
        pl.col('device_0_generic_name') == device
    ).group_by('event_type').agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).collect()
    
    print(f"{'Event Type':<15} {'건수':>10} {'비율':>10}")
    print("-" * 90)
    for event_row in dist.iter_rows(named=True):
        event = event_row['event_type']
        count = event_row['count']
        pct = event_row['percentage']
        print(f"{event:<15} {count:>10,} {pct:>9.1f}%")

Top 3 기기별 Event Type 분포

[1위] AUTOMATED INSULIN DOSING DEVICE SYSTEM, SINGLE HORMONAL CONTROL
총 사건 수: 56,471건
------------------------------------------------------------------------------------------
Event Type              건수         비율
------------------------------------------------------------------------------------------
Malfunction         51,505      91.2%
Injury               4,860       8.6%
Death                  106       0.2%

[2위] TEMPORARY NON-ROLLER TYPE LEFT HEART SUPPORT BLOOD PUMP
총 사건 수: 13,250건
------------------------------------------------------------------------------------------
Event Type              건수         비율
------------------------------------------------------------------------------------------
Injury               7,316      55.2%
Malfunction          4,480      33.8%
Death                1,454      11.0%

[3위] PUMP, INFUSION, INSULIN, TO BE USED WITH INVASIVE GLUCOSE SENSOR
총 사건 수: 11,326건
---------------------------------------------------------

## 일회용/재활용의 피해확인

In [23]:
# 전체 개수
total = df_class3.select(pl.len()).collect().item()

# 분포 계산
reuse_dist = df_class3.group_by('reprocessed_and_reused_flag').agg([
    pl.len().alias('count')
]).with_columns([
    (pl.col('count') / total * 100).round(2).alias('percentage')
]).sort('count', descending=True).collect()

print("=" * 70)
print(f"Reprocessed and Reused Flag 분포 (전체: {total:,}건)")
print("=" * 70)
print(f"{'Flag':<10} {'사건 수':>15} {'비율':>15}")
print("-" * 70)

for row in reuse_dist.iter_rows(named=True):
    flag = row['reprocessed_and_reused_flag'] if row['reprocessed_and_reused_flag'] else "(NULL)"
    count = row['count']
    pct = row['percentage']
    print(f"{flag:<10} {count:>15,} {pct:>14.2f}%")

Reprocessed and Reused Flag 분포 (전체: 186,762건)
Flag                  사건 수              비율
----------------------------------------------------------------------
N                  184,551          98.82%
(NULL)               2,015           1.08%
Y                      196           0.10%


In [24]:
# Reuse Flag × Event Type 크로스탭
crosstab = df_class3.group_by(['reprocessed_and_reused_flag', 'event_type']).agg([
    pl.len().alias('count')
]).sort(['reprocessed_and_reused_flag', 'count'], descending=[False, True]).collect()

print("=== Reuse Flag × Event Type ===")
print(crosstab)

# 또는 각 Flag별로 Event Type 분포
print("\n" + "=" * 70)
for flag in ['Y', 'N', None]:
    flag_display = flag if flag else "(NULL)"
    
    flag_total = df_class3.filter(
        pl.col('reprocessed_and_reused_flag') == flag if flag else pl.col('reprocessed_and_reused_flag').is_null()
    ).select(pl.len()).collect().item()
    
    if flag_total == 0:
        continue
    
    print(f"\n[{flag_display}] 총 {flag_total:,}건")
    print("-" * 70)
    
    event_dist = df_class3.filter(
        pl.col('reprocessed_and_reused_flag') == flag if flag else pl.col('reprocessed_and_reused_flag').is_null()
    ).group_by('event_type').len().sort('len', descending=True).collect()
    
    print(event_dist)

=== Reuse Flag × Event Type ===
shape: (9, 3)
┌─────────────────────────────┬─────────────┬───────┐
│ reprocessed_and_reused_flag ┆ event_type  ┆ count │
│ ---                         ┆ ---         ┆ ---   │
│ str                         ┆ cat         ┆ u32   │
╞═════════════════════════════╪═════════════╪═══════╡
│ null                        ┆ Injury      ┆ 1064  │
│ null                        ┆ Malfunction ┆ 865   │
│ null                        ┆ Death       ┆ 86    │
│ N                           ┆ Malfunction ┆ 99863 │
│ N                           ┆ Injury      ┆ 81089 │
│ N                           ┆ Death       ┆ 3599  │
│ Y                           ┆ Injury      ┆ 100   │
│ Y                           ┆ Malfunction ┆ 92    │
│ Y                           ┆ Death       ┆ 4     │
└─────────────────────────────┴─────────────┴───────┘


[Y] 총 196건
----------------------------------------------------------------------
shape: (3, 2)
┌─────────────┬─────┐
│ event_type  ┆ len │
│ 

# 함수화

event_type 분포 확인

In [25]:
def analyze_column_distribution(df, column_name, top_n=None, show_null=True, verbose=True):
    """
    특정 컬럼의 값 분포와 비율을 분석하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    column_name : str
        분석할 컬럼명
    top_n : int, optional
        상위 N개만 표시 (None이면 전체 표시)
    show_null : bool, default=True
        NULL 값 개수를 별도로 표시할지 여부
    verbose : bool, default=True
        결과를 출력할지 여부
    
    Returns
    -------
    polars.DataFrame
        분포 결과 (컬럼명, count, percentage 포함)
    
    Examples
    --------
    >>> # 기본 사용
    >>> result = analyze_column_distribution(df_class3, 'event_type')
    
    >>> # 상위 10개만
    >>> result = analyze_column_distribution(df_class3, 'device_0_generic_name', top_n=10)
    
    >>> # 출력 없이 결과만
    >>> result = analyze_column_distribution(df_class3, 'event_type', verbose=False)
    """
    # 전체 개수
    total = df.select(pl.len()).collect().item()
    
    # 분포 계산
    dist = df.group_by(column_name).agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True)
    
    # 상위 N개만
    if top_n is not None:
        dist = dist.head(top_n)
    
    result = dist.collect()
    
    # NULL 개수 확인
    null_count = df.filter(pl.col(column_name).is_null()).select(pl.len()).collect().item()
    
    # 출력
    if verbose:
        print("=" * 80)
        print(f"{column_name} 분포 (전체: {total:,}건)")
        if top_n:
            print(f"(상위 {top_n}개만 표시)")
        print("=" * 80)
        print(result)
        
        if show_null:
            print(f"\nNULL 값: {null_count:,}개 ({null_count/total*100:.2f}%)")
    
    return result

In [26]:
# event_type 분포 확인
event_dist = analyze_column_distribution(df_class3, 'event_type')

event_type 분포 (전체: 186,762건)
shape: (3, 3)
┌─────────────┬────────┬────────────┐
│ event_type  ┆ count  ┆ percentage │
│ ---         ┆ ---    ┆ ---        │
│ cat         ┆ u32    ┆ f64        │
╞═════════════╪════════╪════════════╡
│ Malfunction ┆ 100820 ┆ 53.98      │
│ Injury      ┆ 82253  ┆ 44.04      │
│ Death       ┆ 3689   ┆ 1.98       │
└─────────────┴────────┴────────────┘

NULL 값: 0개 (0.00%)


이상 기기 topn

특정 컬럼 기준 상위 n개 출력

In [27]:
def get_top_n_by_column(df, group_column, top_n=10, column_display_name=None, verbose=True):
    """
    특정 컬럼을 기준으로 상위 N개 항목을 추출하고 비율과 함께 출력하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    group_column : str
        그룹화할 컬럼명 (예: 'device_0_generic_name')
    top_n : int, default=10
        상위 N개 항목 추출
    column_display_name : str, optional
        출력 시 표시할 컬럼 이름 (None이면 group_column 사용)
    verbose : bool, default=True
        결과를 출력할지 여부
    
    Returns
    -------
    polars.DataFrame
        상위 N개 항목 (컬럼명, count, percentage 포함)
    
    Examples
    --------
    >>> # 기본 사용 - 상위 20개 기기
    >>> top_devices = get_top_n_by_column(df_class3, 'device_0_generic_name', top_n=20)
    
    >>> # 제조사 상위 10개
    >>> top_manufacturers = get_top_n_by_column(
    ...     df_class3, 
    ...     'device_0_manufacturer_d_name', 
    ...     top_n=10,
    ...     column_display_name='제조사'
    ... )
    
    >>> # 출력 없이 결과만
    >>> result = get_top_n_by_column(df_class3, 'device_0_generic_name', top_n=5, verbose=False)
    """
    # 전체 개수
    total = df.select(pl.len()).collect().item()
    
    # 상위 N개 추출
    top_items = df.group_by(group_column).agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).head(top_n).collect()
    
    # 출력
    if verbose:
        display_name = column_display_name if column_display_name else group_column
        
        print("=" * 90)
        print(f"{display_name} 사건 수 Top {top_n} (전체: {total:,}건)")
        print("=" * 90)
        print()
        
        for i, row in enumerate(top_items.iter_rows(named=True), 1):
            value = row[group_column] if row[group_column] else "(NULL)"
            count = row['count']
            pct = row['percentage']
            
            # 긴 값은 70자로 자르기
            display_value = value[:70] if len(value) > 70 else value
            
            print(f"{i:2d}. {display_value:70s} {count:>8,}건 ({pct:>5.1f}%)")
    
    return top_items

In [28]:
top_devices = get_top_n_by_column(
    df_class3, 
    'device_0_generic_name', 
    top_n=20
)

device_0_generic_name 사건 수 Top 20 (전체: 186,762건)

 1. AUTOMATED INSULIN DOSING DEVICE SYSTEM, SINGLE HORMONAL CONTROL          56,471건 ( 30.2%)
 2. TEMPORARY NON-ROLLER TYPE LEFT HEART SUPPORT BLOOD PUMP                  13,250건 (  7.1%)
 3. PUMP, INFUSION, INSULIN, TO BE USED WITH INVASIVE GLUCOSE SENSOR         11,326건 (  6.1%)
 4. PROSTHESIS, BREAST, NONINFLATABLE, INTERNAL, SILICONE GEL-FILLED          7,781건 (  4.2%)
 5. DEVICE, HEMOSTASIS, VASCULAR                                              6,164건 (  3.3%)
 6. STIMULATOR, SPINAL-CORD, TOTALLY IMPLANTED FOR PAIN RELIEF                5,726건 (  3.1%)
 7. AUTOMATED INSULIN DOSING, THRESHOLD SUSPEND                               5,230건 (  2.8%)
 8. IMPLANTABLE LEAD                                                          5,045건 (  2.7%)
 9. SCS IPG                                                                   5,022건 (  2.7%)
10. SCS LEAD                                                                  4,842건 (  2.6%)
11. NO MAT

topn 기기 event_type 분포 확인

In [29]:
def analyze_top_n_by_category(df, group_column, category_column, top_n=3, 
                               group_display_name=None, category_display_name=None, 
                               verbose=True):
    """
    상위 N개 항목에 대해 카테고리별 분포를 분석하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    group_column : str
        그룹화할 주요 컬럼 (예: 'device_0_generic_name')
    category_column : str
        분포를 확인할 카테고리 컬럼 (예: 'event_type')
    top_n : int, default=3
        상위 N개 항목 분석
    group_display_name : str, optional
        그룹 컬럼의 표시 이름 (None이면 group_column 사용)
    category_display_name : str, optional
        카테고리 컬럼의 표시 이름 (None이면 category_column 사용)
    verbose : bool, default=True
        결과를 출력할지 여부
    
    Returns
    -------
    dict
        각 상위 항목별 카테고리 분포 딕셔너리
    
    Examples
    --------
    >>> # Top 3 기기별 Event Type 분포
    >>> result = analyze_top_n_by_category(
    ...     df_class3, 
    ...     'device_0_generic_name', 
    ...     'event_type',
    ...     top_n=3
    ... )
    
    >>> # Top 5 제조사별 Event Type 분포
    >>> result = analyze_top_n_by_category(
    ...     df_class3,
    ...     'device_0_manufacturer_d_name',
    ...     'event_type',
    ...     top_n=5,
    ...     group_display_name='제조사',
    ...     category_display_name='사건 유형'
    ... )
    
    >>> # Top 10 기기별 재처리 여부 분포
    >>> result = analyze_top_n_by_category(
    ...     df_class3,
    ...     'device_0_generic_name',
    ...     'reprocessed_and_reused_flag',
    ...     top_n=10,
    ...     category_display_name='재처리 여부'
    ... )
    """
    # 상위 N개 항목 추출
    top_items = df.group_by(group_column).len()\
        .sort('len', descending=True).head(top_n).collect()
    
    # 결과 저장용 딕셔너리
    results = {}
    
    # 표시 이름 설정
    group_name = group_display_name if group_display_name else group_column
    category_name = category_display_name if category_display_name else category_column
    
    if verbose:
        print("=" * 90)
        print(f"Top {top_n} {group_name}별 {category_name} 분포")
        print("=" * 90)
    
    # 각 항목에 대해 카테고리 분포 계산
    for rank, row in enumerate(top_items.iter_rows(named=True), 1):
        item_value = row[group_column]
        total_count = row['len']
        
        # 해당 항목의 카테고리 분포
        dist = df.filter(
            pl.col(group_column) == item_value
        ).group_by(category_column).agg([
            pl.len().alias('count')
        ]).with_columns([
            (pl.col('count') / total_count * 100).round(2).alias('percentage')
        ]).sort('count', descending=True).collect()
        
        # 결과 저장
        results[item_value] = dist
        
        # 출력
        if verbose:
            print(f"\n[{rank}위] {item_value}")
            print(f"총 사건 수: {total_count:,}건")
            print("-" * 90)
            print(f"{category_name:<20} {'건수':>15} {'비율':>15}")
            print("-" * 90)
            
            for cat_row in dist.iter_rows(named=True):
                category = cat_row[category_column] if cat_row[category_column] else "(NULL)"
                count = cat_row['count']
                pct = cat_row['percentage']
                print(f"{category:<20} {count:>15,} {pct:>14.1f}%")
    
    if verbose:
        print("=" * 90)
    
    return results

In [30]:
result = analyze_top_n_by_category(
    df_class3,
    'device_0_generic_name',
    'event_type',
    top_n=3
)

Top 3 device_0_generic_name별 event_type 분포

[1위] AUTOMATED INSULIN DOSING DEVICE SYSTEM, SINGLE HORMONAL CONTROL
총 사건 수: 56,471건
------------------------------------------------------------------------------------------
event_type                        건수              비율
------------------------------------------------------------------------------------------
Malfunction                   51,505           91.2%
Injury                         4,860            8.6%
Death                            106            0.2%

[2위] TEMPORARY NON-ROLLER TYPE LEFT HEART SUPPORT BLOOD PUMP
총 사건 수: 13,250건
------------------------------------------------------------------------------------------
event_type                        건수              비율
------------------------------------------------------------------------------------------
Injury                         7,316           55.2%
Malfunction                    4,480           33.8%
Death                          1,454           11.0%

[3위]

In [31]:
# Top 5 제조사의 Event Type 분포
result = analyze_top_n_by_category(
    df_class3,
    'device_0_manufacturer_d_name',
    'event_type',
    top_n=5,
    group_display_name='제조사',
    category_display_name='사건 유형'
)

Top 5 제조사별 사건 유형 분포

[1위] MEDTRONIC PUERTO RICO OPERATIONS CO.
총 사건 수: 72,084건
------------------------------------------------------------------------------------------
사건 유형                             건수              비율
------------------------------------------------------------------------------------------
Malfunction                   65,831           91.3%
Injury                         6,090            8.4%
Death                            163            0.2%

[2위] BOSTON SCIENTIFIC CORPORATION
총 사건 수: 19,914건
------------------------------------------------------------------------------------------
사건 유형                             건수              비율
------------------------------------------------------------------------------------------
Injury                        13,304           66.8%
Malfunction                    6,512           32.7%
Death                             98            0.5%

[3위] ABIOMED, INC.
총 사건 수: 15,250건
---------------------------------------------

In [32]:
# Top 10 기기의 재처리 여부 분포
result = analyze_top_n_by_category(
    df_class3,
    'device_0_generic_name',
    'reprocessed_and_reused_flag',
    top_n=10,
    group_display_name='기기',
    category_display_name='재처리 여부'
)

Top 10 기기별 재처리 여부 분포

[1위] AUTOMATED INSULIN DOSING DEVICE SYSTEM, SINGLE HORMONAL CONTROL
총 사건 수: 56,471건
------------------------------------------------------------------------------------------
재처리 여부                            건수              비율
------------------------------------------------------------------------------------------
N                             56,430           99.9%
Y                                 41            0.1%

[2위] TEMPORARY NON-ROLLER TYPE LEFT HEART SUPPORT BLOOD PUMP
총 사건 수: 13,250건
------------------------------------------------------------------------------------------
재처리 여부                            건수              비율
------------------------------------------------------------------------------------------
N                             13,038           98.4%
(NULL)                           158            1.2%
Y                                 54            0.4%

[3위] PUMP, INFUSION, INSULIN, TO BE USED WITH INVASIVE GLUCOSE SENSOR
총 사건 수: 1

일회용/재사용 기기의 피해 확인

단일 컬럼 분포 분석

In [33]:
def analyze_single_column_pretty(df, column_name, column_display_name=None):
    """
    단일 컬럼의 분포를 예쁘게 출력하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    column_name : str
        분석할 컬럼명
    column_display_name : str, optional
        출력 시 표시할 컬럼 이름 (None이면 column_name 사용)
    
    Returns
    -------
    polars.DataFrame
        분포 결과 (컬럼값, count, percentage 포함)
    
    Examples
    --------
    >>> # Reprocessed Flag 분포
    >>> analyze_single_column_pretty(df_class3, 'reprocessed_and_reused_flag')
    
    >>> # Event Type 분포
    >>> analyze_single_column_pretty(
    ...     df_class3, 
    ...     'event_type',
    ...     column_display_name='사건 유형'
    ... )
    """
    # 전체 개수
    total = df.select(pl.len()).collect().item()
    
    # 분포 계산
    dist = df.group_by(column_name).agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).collect()
    
    # 표시 이름
    display_name = column_display_name if column_display_name else column_name
    
    # 출력
    print("=" * 70)
    print(f"{display_name} 분포 (전체: {total:,}건)")
    print("=" * 70)
    print(f"{'값':<20} {'사건 수':>15} {'비율':>15}")
    print("-" * 70)
    
    for row in dist.iter_rows(named=True):
        value = row[column_name] if row[column_name] else "(NULL)"
        count = row['count']
        pct = row['percentage']
        
        # 긴 값은 자르기
        display_value = value[:18] if len(str(value)) > 18 else value
        print(f"{display_value:<20} {count:>15,} {pct:>14.2f}%")
    
    print("=" * 70)
    
    return dist

두 컬럼 크로스탭 분석

In [34]:
def analyze_crosstab(df, column1, column2, 
                     column1_display_name=None, 
                     column2_display_name=None,
                     show_detail=True):
    """
    두 컬럼의 크로스탭(교차 분석)을 수행하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    column1 : str
        첫 번째 컬럼 (예: 'reprocessed_and_reused_flag')
    column2 : str
        두 번째 컬럼 (예: 'event_type')
    column1_display_name : str, optional
        첫 번째 컬럼의 표시 이름
    column2_display_name : str, optional
        두 번째 컬럼의 표시 이름
    show_detail : bool, default=True
        각 column1 값별로 상세 분포를 출력할지 여부
    
    Returns
    -------
    polars.DataFrame
        크로스탭 결과
    
    Examples
    --------
    >>> # Reuse Flag × Event Type
    >>> analyze_crosstab(
    ...     df_class3, 
    ...     'reprocessed_and_reused_flag', 
    ...     'event_type'
    ... )
    
    >>> # 제조사 × Event Type
    >>> analyze_crosstab(
    ...     df_class3,
    ...     'device_0_manufacturer_d_name',
    ...     'event_type',
    ...     column1_display_name='제조사',
    ...     column2_display_name='사건 유형'
    ... )
    
    >>> # 크로스탭만 보기 (상세 분포 제외)
    >>> analyze_crosstab(
    ...     df_class3,
    ...     'reprocessed_and_reused_flag',
    ...     'event_type',
    ...     show_detail=False
    ... )
    """
    # 표시 이름 설정
    col1_name = column1_display_name if column1_display_name else column1
    col2_name = column2_display_name if column2_display_name else column2
    
    # 크로스탭 생성
    crosstab = df.group_by([column1, column2]).agg([
        pl.len().alias('count')
    ]).sort([column1, 'count'], descending=[False, True]).collect()
    
    print("=" * 90)
    print(f"{col1_name} × {col2_name} 크로스탭")
    print("=" * 90)
    print(crosstab)
    
    # 상세 분포 출력
    if show_detail:
        print("\n" + "=" * 90)
        print(f"{col1_name}별 {col2_name} 상세 분포")
        print("=" * 90)
        
        # column1의 고유값 추출 (NULL 포함)
        unique_values = df.select(column1).unique().collect()[column1].to_list()
        
        # NULL도 포함
        if None not in unique_values and df.filter(pl.col(column1).is_null()).select(pl.len()).collect().item() > 0:
            unique_values.append(None)
        
        for value in unique_values:
            value_display = value if value else "(NULL)"
            
            # 해당 값의 총 개수
            value_total = df.filter(
                pl.col(column1) == value if value else pl.col(column1).is_null()
            ).select(pl.len()).collect().item()
            
            if value_total == 0:
                continue
            
            print(f"\n[{value_display}] 총 {value_total:,}건")
            print("-" * 90)
            
            # 해당 값의 column2 분포
            dist = df.filter(
                pl.col(column1) == value if value else pl.col(column1).is_null()
            ).group_by(column2).agg([
                pl.len().alias('count')
            ]).with_columns([
                (pl.col('count') / value_total * 100).round(2).alias('percentage')
            ]).sort('count', descending=True).collect()
            
            print(f"{col2_name:<25} {'건수':>15} {'비율':>15}")
            print("-" * 90)
            
            for row in dist.iter_rows(named=True):
                cat = row[column2] if row[column2] else "(NULL)"
                count = row['count']
                pct = row['percentage']
                
                cat_display = cat[:23] if len(str(cat)) > 23 else cat
                print(f"{cat_display:<25} {count:>15,} {pct:>14.2f}%")
        
        print("=" * 90)
    
    return crosstab

일회용/재사용의 사건 분포 비교

In [35]:
reuse_dist = analyze_single_column_pretty(
    df_class3, 
    'reprocessed_and_reused_flag',
    column_display_name='재처리/재사용 여부'
)

재처리/재사용 여부 분포 (전체: 186,762건)
값                               사건 수              비율
----------------------------------------------------------------------
N                            184,551          98.82%
(NULL)                         2,015           1.08%
Y                                196           0.10%


reuse flag X event_type 

In [36]:
crosstab_result = analyze_crosstab(
    df_class3,
    'reprocessed_and_reused_flag',
    'event_type',
    column1_display_name='재처리 여부',
    column2_display_name='사건 유형'
)

재처리 여부 × 사건 유형 크로스탭
shape: (9, 3)
┌─────────────────────────────┬─────────────┬───────┐
│ reprocessed_and_reused_flag ┆ event_type  ┆ count │
│ ---                         ┆ ---         ┆ ---   │
│ str                         ┆ cat         ┆ u32   │
╞═════════════════════════════╪═════════════╪═══════╡
│ null                        ┆ Injury      ┆ 1064  │
│ null                        ┆ Malfunction ┆ 865   │
│ null                        ┆ Death       ┆ 86    │
│ N                           ┆ Malfunction ┆ 99863 │
│ N                           ┆ Injury      ┆ 81089 │
│ N                           ┆ Death       ┆ 3599  │
│ Y                           ┆ Injury      ┆ 100   │
│ Y                           ┆ Malfunction ┆ 92    │
│ Y                           ┆ Death       ┆ 4     │
└─────────────────────────────┴─────────────┴───────┘

재처리 여부별 사건 유형 상세 분포

[Y] 총 196건
------------------------------------------------------------------------------------------
사건 유형                           

## 치명률 비교

함수화

In [37]:
def calculate_cfr_by_device(df, device_column='device_0_generic_name', 
                            event_column='event_type',
                            top_n=None, min_cases=10):
    """
    기기별 치명률(Case Fatality Rate)을 계산하는 함수
    
    치명률(CFR) = (사망 건수 / 해당 기기 총 보고 건수) × 100
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    device_column : str, default='device_0_generic_name'
        기기 컬럼명
    event_column : str, default='event_type'
        사건 유형 컬럼명
    top_n : int, optional
        상위 N개 기기만 분석 (None이면 전체)
    min_cases : int, default=10
        최소 보고 건수 (이보다 적은 기기는 제외, 통계적 신뢰도 확보)
    
    Returns
    -------
    polars.DataFrame
        기기별 치명률 결과 (기기명, 총건수, 사망건수, 부상건수, 오작동건수, CFR)
    
    Examples
    --------
    >>> # 전체 기기 CFR
    >>> cfr_result = calculate_cfr_by_device(df_class3)
    
    >>> # 상위 20개 기기만
    >>> cfr_top20 = calculate_cfr_by_device(df_class3, top_n=20)
    
    >>> # 최소 100건 이상 보고된 기기만
    >>> cfr_reliable = calculate_cfr_by_device(df_class3, min_cases=100)
    """
    
    # 기기별 전체 건수와 사건 유형별 건수
    device_stats = df.group_by(device_column).agg([
        pl.len().alias('total_cases'),
        (pl.col(event_column) == 'Death').sum().alias('death_count'),
        (pl.col(event_column) == 'Injury').sum().alias('injury_count'),
        (pl.col(event_column) == 'Malfunction').sum().alias('malfunction_count')
    ]).filter(
        pl.col('total_cases') >= min_cases  # 최소 건수 필터
    ).with_columns([
        # CFR 계산
        (pl.col('death_count') / pl.col('total_cases') * 100).round(2).alias('cfr'),
        # 부상률
        (pl.col('injury_count') / pl.col('total_cases') * 100).round(2).alias('injury_rate'),
        # 오작동률
        (pl.col('malfunction_count') / pl.col('total_cases') * 100).round(2).alias('malfunction_rate')
    ]).sort('cfr', descending=True)
    
    # Top N만
    if top_n:
        device_stats = device_stats.head(top_n)
    
    result = device_stats.collect()
    
    # 출력
    print("=" * 120)
    print(f"기기별 치명률(CFR) 분석 (최소 {min_cases}건 이상)")
    print("=" * 120)
    print(f"{'순위':>4} {'기기명':<50} {'총건수':>10} {'사망':>8} {'부상':>8} {'오작동':>10} {'CFR(%)':>10}")
    print("-" * 120)
    
    for i, row in enumerate(result.iter_rows(named=True), 1):
        device = row[device_column] if row[device_column] else "(NULL)"
        total = row['total_cases']
        death = row['death_count']
        injury = row['injury_count']
        mal = row['malfunction_count']
        cfr = row['cfr']
        
        device_short = device[:48] if len(device) > 48 else device
        
        print(f"{i:4d} {device_short:<50} {total:>10,} {death:>8,} {injury:>8,} {mal:>10,} {cfr:>10.2f}%")
    
    print("=" * 120)
    
    # 요약 통계
    print(f"\n◼️ 요약 통계:")
    print(f"  - 분석 기기 수: {len(result):,}개")
    print(f"  - 평균 CFR: {result['cfr'].mean():.2f}%")
    print(f"  - 최대 CFR: {result['cfr'].max():.2f}%")
    print(f"  - 최소 CFR: {result['cfr'].min():.2f}%")
    print(f"  - CFR 중앙값: {result['cfr'].median():.2f}%")
    
    return result

In [38]:
# 최소 10건 이상 보고된 기기의 CFR
cfr_all = calculate_cfr_by_device(df_class3, top_n=20)

기기별 치명률(CFR) 분석 (최소 10건 이상)
  순위 기기명                                                       총건수       사망       부상        오작동     CFR(%)
------------------------------------------------------------------------------------------------------------------------
   1 PERCUTANEOUSLY DELIVERED PROSTHESES AND TRICUSPI           16        5       11          0      31.25%
   2 Ventricular (assist) bypass                               271       76       71        124      28.04%
   3 VENTRICULAR (ASSIST) DEVICE                                28        7        7         14      25.00%
   4 SHOCKWAVE C2+ CORONARY INTRAVASCULAR LITHOTRIPSY           27        6       20          1      22.22%
   5 VENTRICULAR (ASSIST) BYPASS                             4,404      944    1,455      2,005      21.44%
   6 NEVRO SENZA                                               842      143      678         21      16.98%
   7 TEMPORARY NON-ROLLER TYPE RIGHT HEART SUPPORT BL          874      117      397        360 

제조사별 CFR

In [39]:
# 제조사별 치명률
cfr_by_manufacturer = calculate_cfr_by_device(
    df_class3, 
    device_column='device_0_manufacturer_d_name',
    min_cases=50
)

기기별 치명률(CFR) 분석 (최소 50건 이상)
  순위 기기명                                                       총건수       사망       부상        오작동     CFR(%)
------------------------------------------------------------------------------------------------------------------------
   1 ABIOMED. INC.                                             166       41       84         41      24.70%
   2 THORATEC CORPORATION                                    4,720    1,032    1,538      2,150      21.86%
   3 NEVRO CORP.                                               842      143      678         21      16.98%
   4 EDWARDS LIFESCIENCES                                      399       50      185        164      12.53%
   5 MEDTRONIC HEART VALVES DIVISION                           248       27      110        111      10.89%
   6 ABIOMED, INC.                                          15,250    1,599    7,269      6,382      10.49%
   7 CARDIOVASCULAR SYSTEMS, INC.                               79        8       64          7 

## CFR 구간별 분포

In [40]:
def analyze_cfr_distribution(df, device_column='device_0_generic_name', min_cases=10):
    """
    CFR 구간별 기기 분포를 분석하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    device_column : str
        기기 컬럼명
    min_cases : int
        최소 보고 건수
    
    Examples
    --------
    >>> analyze_cfr_distribution(df_class3)
    """
    # CFR 계산
    cfr_data = df.group_by(device_column).agg([
        pl.len().alias('total_cases'),
        (pl.col('event_type') == 'Death').sum().alias('death_count')
    ]).filter(
        pl.col('total_cases') >= min_cases
    ).with_columns([
        (pl.col('death_count') / pl.col('total_cases') * 100).alias('cfr')
    ]).collect()
    
    # CFR 구간별 분류
    cfr_ranges = [
        (0, 1, "매우 낮음 (0-1%)"),
        (1, 3, "낮음 (1-3%)"),
        (3, 5, "보통 (3-5%)"),
        (5, 10, "높음 (5-10%)"),
        (10, 100, "매우 높음 (10%+)")
    ]
    
    print("=" * 80)
    print("CFR 구간별 기기 분포")
    print("=" * 80)
    print(f"{'CFR 구간':<25} {'기기 수':>15} {'비율':>15}")
    print("-" * 80)
    
    total_devices = len(cfr_data)
    
    for min_cfr, max_cfr, label in cfr_ranges:
        count = cfr_data.filter(
            (pl.col('cfr') >= min_cfr) & (pl.col('cfr') < max_cfr)
        ).shape[0]
        
        pct = (count / total_devices * 100) if total_devices > 0 else 0
        print(f"{label:<25} {count:>15,} {pct:>14.1f}%")
    
    print("=" * 80)
    print(f"{'총 기기 수':<25} {total_devices:>15,} {100.0:>14.1f}%")
    print("=" * 80)

In [41]:
# CFR 구간별 기기 분포
analyze_cfr_distribution(df_class3, min_cases=50)

CFR 구간별 기기 분포
CFR 구간                               기기 수              비율
--------------------------------------------------------------------------------
매우 낮음 (0-1%)                          103           71.0%
낮음 (1-3%)                              18           12.4%
보통 (3-5%)                               8            5.5%
높음 (5-10%)                             10            6.9%
매우 높음 (10%+)                            6            4.1%
총 기기 수                                145          100.0%


특정 기기의 상세 분석

In [42]:
def analyze_device_detail(df, device_name, device_column='device_0_generic_name'):
    """
    특정 기기의 상세 통계 분석
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    device_name : str
        분석할 기기명
    device_column : str
        기기 컬럼명
    
    Examples
    --------
    >>> analyze_device_detail(df_class3, 'Catheter, Intravascular, Therapeutic')
    """
    device_data = df.filter(pl.col(device_column) == device_name).collect()
    
    total = len(device_data)
    death = device_data.filter(pl.col('event_type') == 'Death').shape[0]
    injury = device_data.filter(pl.col('event_type') == 'Injury').shape[0]
    mal = device_data.filter(pl.col('event_type') == 'Malfunction').shape[0]
    
    cfr = (death / total * 100) if total > 0 else 0
    injury_rate = (injury / total * 100) if total > 0 else 0
    mal_rate = (mal / total * 100) if total > 0 else 0
    
    print("=" * 80)
    print(f"기기 상세 분석: {device_name}")
    print("=" * 80)
    print(f"\n◼️ 기초 통계:")
    print(f"  총 보고 건수: {total:,}건")
    print(f"\n◼️ 사건 유형별 분포:")
    print(f"  • 사망 (Death):        {death:>8,}건 ({cfr:>5.2f}%) ⚠️ CFR")
    print(f"  • 부상 (Injury):       {injury:>8,}건 ({injury_rate:>5.2f}%)")
    print(f"  • 오작동 (Malfunction): {mal:>8,}건 ({mal_rate:>5.2f}%)")
    print("=" * 80)

In [43]:
# 특정 기기 상세 분석
analyze_device_detail(df_class3, 'Ventricular (assist) bypass')

기기 상세 분석: Ventricular (assist) bypass

◼️ 기초 통계:
  총 보고 건수: 271건

◼️ 사건 유형별 분포:
  • 사망 (Death):              76건 (28.04%) ⚠️ CFR
  • 부상 (Injury):             71건 (26.20%)
  • 오작동 (Malfunction):      124건 (45.76%)
